In [1]:
%%bash
pip install pyyaml -q

In [2]:
import os
import cv2
import yaml
import random as r
import numpy as np
import pandas as pd
from yaml.loader import SafeLoader

BASE_PATH: str = "../input/playing-cards-object-detection-dataset"
    
def breaker(num: int=50, char: str="*") -> None:
    print("\n" + num*char + "\n")

In [3]:
for folder_name in ["train", "valid", "test"]:

    img_filenames = [filename[:-4] for filename in sorted(os.listdir(BASE_PATH + f"/{folder_name}/images"))]
    txt_filenames = [filename[:-4] for filename in sorted(os.listdir(BASE_PATH + f"/{folder_name}/labels"))]

    assert img_filenames == txt_filenames, f"{folder_name} Failure"

In [4]:
with open("../input/playing-cards-object-detection-dataset/data.yaml") as f:
    yaml_data = yaml.load(f, Loader=SafeLoader)
class_names = yaml_data["names"]

In [5]:
# with open("../input/playing-cards-object-detection-dataset/train/labels/000090528_jpg.rf.d50e89610e5c97c61632c290692f3e75.txt") as f:

path = "../input/playing-cards-object-detection-dataset/train/labels"
name = os.listdir(path)[r.randint(0, len(os.listdir(path))-1)]

with open(os.path.join(path, name)) as f:
    data = f.read()
    
data = [data_item.split(" ") for data_item in data.split("\n")]
data

[['22',
  '0.41947115384615385',
  '0.4579326923076923',
  '0.06129807692307692',
  '0.04567307692307692'],
 ['26',
  '0.43990384615384615',
  '0.5096153846153846',
  '0.06610576923076923',
  '0.04447115384615385'],
 ['12',
  '0.44711538461538464',
  '0.6057692307692307',
  '0.06850961538461539',
  '0.03365384615384615'],
 ['12', '0.234375', '0.78125', '0.06850961538461539', '0.03245192307692308']]

In [6]:
for folder_name in ["train", "valid", "test"]:

    df = pd.DataFrame(data=None, columns=["names", "classes", "bounding_boxes"])
    path = f"../input/playing-cards-object-detection-dataset/{folder_name}/labels"
    names = np.array([name[:-4] for name in sorted(os.listdir(path))])

    final_classes: list = []
    final_bboxes : list = []

    for name in names:
        classes: list = []
        bboxes: list  = []
        with open(os.path.join(path, name + ".txt")) as f:
            data = f.read()

        data = [data_item.split(" ") for data_item in data.split("\n")]

        for item in data:
            classes.append(int(item[0]))
            bboxes.append([
                float(item[1]), 
                float(item[2]), 
                float(item[3]), 
                float(item[4]),
            ])

        final_classes.append(classes)
        final_bboxes.append(bboxes) 
        
    df.split = folder_name
    df.names = names
    df.classes = final_classes
    df.bounding_boxes = final_bboxes
    df.to_csv(f"{folder_name}.csv", index=False)

### **Dataset and DataLoader Analysis**

In [7]:
import torch

from torch.utils.data import Dataset
from torch.utils.data import DataLoader as DL

from torchvision import transforms
from torch.nn.utils.rnn import pad_sequence

In [8]:
def get_image(path: str, size: int=224) -> np.ndarray:
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2RGB)
    image = cv2.resize(src=image, dsize=(size, size), interpolation=cv2.INTER_AREA)
    return image

In [9]:
class DS(Dataset):
    def __init__(
        self, 
        base_path: str="../input/playing-cards-object-detection-dataset/valid/images",
        filenames: np.ndarray=None, 
        labels: list=None, 
        bboxes: list=None,
        size: int=224,
        transform: transforms=transforms.Compose([transforms.ToTensor(),])
    ):
        self.base_path = base_path
        self.filenames = filenames
        self.labels = labels
        self.bboxes = bboxes
        self.size = size
        self.transform = transform
    
    def __len__(self):
        return self.filenames.shape[0]

    def __getitem__(self, idx):
        image = get_image(os.path.join(self.base_path, self.filenames[idx] + ".jpg"), self.size)
        if self.transform:
            image = self.transform(image)
        label = torch.LongTensor(self.labels[idx])
        bbox  = torch.FloatTensor(self.bboxes[idx])
        
        return image, label, bbox

In [10]:
def my_collate_fn(batch):
    label_list: list = []
    bbox_list: list  = []
    images = torch.FloatTensor(len(batch), 3, 224, 224)
    
    i = 0
    for image_, label_, box_ in batch:
        images[i] = image_
        label_list.append(label_)
        bbox_list.append(box_)       
    
    label_list = pad_sequence(label_list, batch_first=True, padding_value=53)
    bbox_list = pad_sequence(bbox_list, batch_first=True, padding_value=0)

    return images, label_list, bbox_list

In [11]:
path = f"../input/playing-cards-object-detection-dataset/valid/labels"
names = np.array([name[:-4] for name in sorted(os.listdir(path))])

va_data_setup = DS(filenames=names, labels=final_classes, bboxes=final_bboxes)
va_data = DL(va_data_setup, batch_size=16, shuffle=False, collate_fn=my_collate_fn)

In [12]:
imgs, lbls, bbxs = next(iter(va_data))

print(f"imgs.shape    : {imgs.shape}")
print(f"imgs[0].shape : {imgs[0].shape}")
print(f"lbls.shape    : {lbls.shape}")
print(f"bbxs.shape    : {bbxs.shape}")

imgs.shape    : torch.Size([16, 3, 224, 224])
imgs[0].shape : torch.Size([3, 224, 224])
lbls.shape    : torch.Size([16, 4])
bbxs.shape    : torch.Size([16, 4, 4])


### **Statistics**

In [13]:
def get_statistics(path: str) -> dict:

    sizes = [224, 384, 512]
    stats: dict = dict()
    means: dict = dict()
    stds: dict  = dict()

    for size in sizes:
        r_mean, g_mean, b_mean, r_std, g_std, b_std = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
        for name in os.listdir(path):
            image = get_image(os.path.join(path, name), size)
            r_mean += image[:, :, 0].mean()
            g_mean += image[:, :, 1].mean()
            b_mean += image[:, :, 2].mean()
            r_std += image[:, :, 0].std()
            g_std += image[:, :, 1].std()
            b_std += image[:, :, 2].std()

        means[f"{size}"] = [
            r_mean / len(os.listdir(path)) / 255, 
            g_mean / len(os.listdir(path)) / 255, 
            b_mean / len(os.listdir(path)) / 255,
        ]
        stds[f"{size}"]  = [
            r_std / len(os.listdir(path)) / 255, 
            g_std / len(os.listdir(path)) / 255, 
            b_std / len(os.listdir(path)) / 255,
        ]

    stats["means"] = means
    stats["stds"] = stds
    
    return stats


def print_statistics(stats: dict) -> None:
    breaker()
    print("Means".upper())
    breaker()
    for k, v in stats["means"].items():
        print(f"{k} x {k} Means\n")
        for i in range(len(v)):
            if i == 0:
                print(f"Red   : {v[i]:.5f}")
            if i == 1:
                print(f"Green : {v[i]:.5f}")
            if i == 2:
                print(f"Blue  : {v[i]:.5f}")
        breaker()

    print("\n\n\n")

    breaker()
    print("Stds".upper())
    breaker()
    for k, v in stats["stds"].items():
        print(f"{k} x {k} Stds\n")
        for i in range(len(v)):
            if i == 0:
                print(f"Red   : {v[i]:.5f}")
            if i == 1:
                print(f"Green : {v[i]:.5f}")
            if i == 2:
                print(f"Blue  : {v[i]:.5f}")
        breaker()

**Train**

In [14]:
path = "../input/playing-cards-object-detection-dataset/train/images"

stats = get_statistics(path)
print_statistics(stats)


**************************************************

MEANS

**************************************************

224 x 224 Means

Red   : 0.55440
Green : 0.49272
Blue  : 0.44704

**************************************************

384 x 384 Means

Red   : 0.55440
Green : 0.49272
Blue  : 0.44704

**************************************************

512 x 512 Means

Red   : 0.55405
Green : 0.49238
Blue  : 0.44669

**************************************************






**************************************************

STDS

**************************************************

224 x 224 Stds

Red   : 0.19316
Green : 0.19301
Blue  : 0.19653

**************************************************

384 x 384 Stds

Red   : 0.19614
Green : 0.19604
Blue  : 0.19948

**************************************************

512 x 512 Stds

Red   : 0.19712
Green : 0.19704
Blue  : 0.20045

**************************************************



**Valid**

In [15]:
path = "../input/playing-cards-object-detection-dataset/valid/images"

stats = get_statistics(path)
print_statistics(stats)


**************************************************

MEANS

**************************************************

224 x 224 Means

Red   : 0.55624
Green : 0.49625
Blue  : 0.45093

**************************************************

384 x 384 Means

Red   : 0.55624
Green : 0.49625
Blue  : 0.45093

**************************************************

512 x 512 Means

Red   : 0.55589
Green : 0.49590
Blue  : 0.45059

**************************************************






**************************************************

STDS

**************************************************

224 x 224 Stds

Red   : 0.19386
Green : 0.19374
Blue  : 0.19746

**************************************************

384 x 384 Stds

Red   : 0.19695
Green : 0.19689
Blue  : 0.20052

**************************************************

512 x 512 Stds

Red   : 0.19797
Green : 0.19792
Blue  : 0.20152

**************************************************



**Test**

In [16]:
path = "../input/playing-cards-object-detection-dataset/test/images"

stats = get_statistics(path)
print_statistics(stats)


**************************************************

MEANS

**************************************************

224 x 224 Means

Red   : 0.55492
Green : 0.49802
Blue  : 0.45294

**************************************************

384 x 384 Means

Red   : 0.55492
Green : 0.49802
Blue  : 0.45294

**************************************************

512 x 512 Means

Red   : 0.55458
Green : 0.49767
Blue  : 0.45260

**************************************************






**************************************************

STDS

**************************************************

224 x 224 Stds

Red   : 0.19541
Green : 0.19503
Blue  : 0.19883

**************************************************

384 x 384 Stds

Red   : 0.19837
Green : 0.19806
Blue  : 0.20179

**************************************************

512 x 512 Stds

Red   : 0.19935
Green : 0.19905
Blue  : 0.20276

**************************************************

